In [1]:
!pip install transformers torch h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=a45dd5c2763c3620e43d4306c8e4a4696f1c85617c461869503b69b1f270e8c9
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Start H2O server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2rtvn922
  JVM stdout: /tmp/tmp2rtvn922/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp2rtvn922/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_5jduy9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
data_path = 'data.csv'
data = pd.read_csv(data_path)

# Convert to H2O Frame for further processing
hf = h2o.H2OFrame(data)

# Display basic info and check for missing values
print(hf.columns)
print(hf.describe())

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['countries', 'year', 'Risk_Probability_both', 'Risk_Probability_Male', 'Risk_Probability_Female']


Rows:3660
Cols:5

,countries,year,Risk_Probability_both,Risk_Probability_Male,Risk_Probability_Female
type,enum,int,real,real,real
mins,,2000.0,7.3,9.6,4.4
mean,,2009.4999999999982,22.058415300546386,25.63959016393442,18.6704644808743
maxs,,2019.0,56.0,64.1,47.8
sigma,,5.767069202141596,8.170808578108995,9.727200775328123,7.530725206780123
zeros,,0,0,0,0
missing,0,0,0,0,0
0,Afghanistan,2019.0,35.3,34.4,36.2
1,Afghanistan,2018.0,35.4,34.7,36.0
2,Afghanistan,2017.0,35.5,35.0,35.9


None


In [5]:
# Handle missing values
hf = hf.na_omit()  # Drop rows with missing values

# Convert target column to numeric (if needed) and categorize as high or low risk
hf["Risk_Probability_both"] = hf["Risk_Probability_both"].asnumeric()

# Create a boolean column where 'True' means "High Risk" and 'False' means "Low Risk"
hf["Risk_Flag"] = hf["Risk_Probability_both"] > 30

# Convert the boolean column to "High Risk" and "Low Risk"
hf["Risk_Level"] = hf["Risk_Flag"].ifelse("High Risk", "Low Risk")


In [6]:
# Convert the target to a factor for classification
hf['Risk_Level'] = hf['Risk_Level'].asfactor()

# Define predictor columns and remove the target column from predictors
predictors = hf.columns
predictors.remove('Risk_Level')

In [7]:
# Split the dataset into training and testing sets
train, test = hf.split_frame(ratios=[0.8], seed=1234)

In [8]:
# Initialize H2O AutoML with desired parameters
aml = H2OAutoML(max_models=20, seed=1, balance_classes=True, sort_metric='AUC')
aml.train(x=predictors, y='Risk_Level', training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_1_20241108_80526_model_1


Model Summary: 
    number_of_trees
--  -----------------
    50

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 3.329583646452904e-05
RMSE: 0.005770254454053915
LogLoss: 0.0040368292707528885
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9729368686676025
           Low Risk    High Risk    Error    Rate
---------  ----------  -----------  -------  ------------
Low Risk   2521        0            0        (0.0/2521.0)
High Risk  0           428          0        (0.0/428.0)
Total      2521        428          0        (0.0/2949.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.972937     1         12
max f2                       0.972937     1         12
max f0point5                 0.972937     1         12
max accuracy                 0.972937     1         12
max precision                0.989184     1         0
max recall                   0.972937     1         12
max specificity              0.989184     1         0
max absolute_mcc             0.972937     1         12
max min_per_class_accuracy   0.972937     1         12
max mean_per_class_accuracy  0.972937     1         12
max tns                      0.989184     2521      0
max fns                      0.989184     140       0
max fps                      0.00211353   2521      42
max tps                      0.972937     428       12
max tnr                      0.989184     1         0
max fnr                      0.989184     0.327103  0
max fpr                      0.00211353   1         42
max tpr                      0.972937     1         12

Gains/Lift Table: Avg response rate: 14.51 %, avg score: 14.53 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0976602                   0.989184           6.89019  6.89019            1                0.989184    1                           0.989184            0.672897        0.672897                   589.019  589.019            0.672897
2        0.101051                    0.98778            6.89019  6.89019            1                0.98778     1                           0.989137            0.0233645       0.696262                   589.019  589.019            0.696262
3        0.150899                    0.0073585          6.09336  6.62697            0.884354         0.868904    0.961798                    0.94942             0.303738        1                          509.336  562.697            0.993257
4        0.223466                    0.00374327         0        4.47496            0                0.00465198  0.649469                    0.642621            0               1                          -100     347.496            0.90837
5        0.30078                     0.0027702          0        3.32469            0                0.00305455  0.482525                    0.478223            0               1                          -100     232.469            0.817929
6        1                           0.00211353         0        1                  0                0.00212899  0.145134                    0.145328            0               1                          -100     0                  0

ModelMetricsBinomial: xgboost
** Reported on cross-validation data. **

MSE: 5.125795850

In [9]:
# Display leaderboard of models
lb = aml.leaderboard
print(lb)

model_id                                               auc      logloss    aucpr    mean_per_class_error         rmse          mse
XGBoost_grid_1_AutoML_1_20241108_80526_model_1           1  0.00510979         1                       0  0.00715947   5.1258e-05
GBM_4_AutoML_1_20241108_80526                            1  3.96126e-16        1                       0  1.70893e-14  2.92043e-28
GBM_grid_1_AutoML_1_20241108_80526_model_2               1  6.21183e-18        1                       0  2.90345e-16  8.43005e-32
XGBoost_1_AutoML_1_20241108_80526                        1  0.00668347         1                       0  0.00701295   4.91815e-05
XRT_1_AutoML_1_20241108_80526                            1  0.029372           1                       0  0.0440043    0.00193638
XGBoost_2_AutoML_1_20241108_80526                        1  0.00359424         1                       0  0.00376789   1.4197e-05
GLM_1_AutoML_1_20241108_80526                            1  0.00714172         1      

In [10]:
# Select the best model based on leaderboard
best_model = aml.leader
print(best_model.model_id)

XGBoost_grid_1_AutoML_1_20241108_80526_model_1


In [11]:
# Make predictions on the test set
predictions = best_model.predict(test)
print(predictions)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
predict      Low Risk    High Risk
High Risk   0.0108155   0.989184
High Risk   0.0108155   0.989184
High Risk   0.0108155   0.989184
High Risk   0.0108155   0.989184
High Risk   0.0108155   0.989184
High Risk   0.0108155   0.989184
Low Risk    0.997886    0.00211353
Low Risk    0.997886    0.00211353
Low Risk    0.997886    0.00211353
Low Risk    0.997886    0.00211353
[711 rows x 3 columns]



In [12]:
# Evaluate model performance
performance = best_model.model_performance(test)
print(performance)

ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 3.576221792487651e-05
RMSE: 0.005980151998476001
LogLoss: 0.004171170158637125
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9761784672737122
           Low Risk    High Risk    Error    Rate
---------  ----------  -----------  -------  -----------
Low Risk   601         0            0        (0.0/601.0)
High Risk  0           110          0        (0.0/110.0)
Total      601         110          0        (0.0/711.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.976178     1         11
max f2                       0.976178     1         11
max f0point5                 0.976178     1         11
max accuracy                 0.976178     1         11
max precision                0.989184     1       

In [13]:
# Forward Chaining
def forward_chaining(symptoms):
    """Incorporates forward chaining rules to infer potential diseases based on symptoms."""
    if 'chest_pain' in symptoms and 'cholesterol' in symptoms:
        if symptoms['chest_pain'] and symptoms['cholesterol'] > 200:
            return 'High Risk of Cardiovascular Disease'
    return 'Low Risk of Cardiovascular Disease'

In [14]:
# Backward Chaining
def backward_chaining(diagnosis):
    """Uses backward chaining to verify conditions."""
    if diagnosis == 'High Risk of Cardiovascular Disease':
        if 'chest_pain' in hf.columns and 'cholesterol' in hf.columns:
            if hf['chest_pain'].any() and hf['cholesterol'].mean() > 200:
                return True
    return False


In [15]:
# Treatment Recommendation
def treatment_recommendation(diagnosis):
    """Provide recommendations based on the diagnosis."""
    if diagnosis == 'High Risk of Cardiovascular Disease':
        return "Recommendation: Consult a cardiologist, adopt a heart-healthy diet, and start regular exercise."
    else:
        return "Recommendation: Maintain a healthy lifestyle and schedule regular check-ups."


In [16]:
#  Load the pre-trained BERT model for text classification
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Analyze symptoms using BERT
def analyze_symptoms_with_bert(text_input):
    """
    Uses BERT to classify user symptoms or feedback.
    Args:
        text_input (str): Free-form text describing symptoms.

    Returns:
        (str): Predicted class label indicating high or low cardiovascular risk.
    """
    # Tokenize input text and convert to PyTorch tensors
    inputs = tokenizer(text_input, return_tensors="pt", max_length=128, padding='max_length', truncation=True)

    # Perform forward pass and obtain predictions
    outputs = bert_model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    if predicted_class == 1:  # Assuming 1 indicates high-risk condition
        return "High Risk of Cardiovascular Disease"
    else:
        return "Low Risk of Cardiovascular Disease"

In [18]:
# Enhanced Diagnosis combining AutoML and BERT
def enhanced_diagnosis(user_data, text_description):
    """
    Combines symptom analysis from BERT with AutoML-based risk prediction.

    Args:
        user_data (dict): User-provided structured data (e.g., symptoms, cholesterol levels).
        text_description (str): Free-form text description of symptoms or additional information.

    Returns:
        (str, str): Diagnosis and treatment recommendation.
    """
    # Step 1: Get a diagnosis from AutoML (using forward/backward chaining as in the earlier implementation)
    initial_diagnosis = forward_chaining(user_data)
    if backward_chaining(initial_diagnosis):
        diagnosis = initial_diagnosis
    else:
        diagnosis = "Uncertain"

    # Step 2: Use BERT to further refine the diagnosis based on text input
    bert_diagnosis = analyze_symptoms_with_bert(text_description)

    # Step 3: Combine results and finalize treatment recommendations
    if bert_diagnosis == "High Risk of Cardiovascular Disease" or diagnosis == "High Risk of Cardiovascular Disease":
        final_diagnosis = "High Risk of Cardiovascular Disease"
        recommendation = treatment_recommendation(final_diagnosis)
    else:
        final_diagnosis = "Low Risk of Cardiovascular Disease"
        recommendation = treatment_recommendation(final_diagnosis)

    return final_diagnosis, recommendation


In [1]:
import gradio as gr
import h2o
from transformers import BertTokenizer, BertForSequenceClassification
import json

# Initialize H2O server
h2o.init()

# Load your pre-trained BERT model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

def predict_risk(chest_pain, smoking, cholesterol, age, blood_pressure, symptoms_description):
    try:
        # Calculate risk score based on various factors
        cholesterol_risk = 1 if cholesterol >= 240 else 0  # High-risk if cholesterol >= 240 mg/dL
        blood_pressure_risk = 1 if blood_pressure >= 140 else 0  # High-risk if blood pressure >= 140/90

        # Calculate a risk score based on factors
        risk_score = cholesterol_risk + blood_pressure_risk + (age / 60) + (1 if chest_pain else 0) + (1 if smoking else 0)

        # Diagnosis based on risk score
        diagnosis = "High Risk" if risk_score > 2 else "Low Risk"
        recommendation = "Consult a healthcare provider" if diagnosis == "High Risk" else "Maintain a healthy lifestyle"

        return diagnosis, recommendation

    except Exception as e:
        return f"An error occurred: {str(e)}", ""

# Define the Gradio interface
iface = gr.Interface(
    fn=predict_risk,
    inputs=[
        gr.Checkbox(label="Chest Pain", value=False),  # Chest pain as checkbox
        gr.Checkbox(label="Smoking", value=False),  # Smoking as checkbox
        gr.Slider(label="Cholesterol", minimum=100, maximum=400, step=1, value=200),  # Cholesterol as slider
        gr.Slider(label="Age", minimum=18, maximum=100, step=1, value=30),  # Age as slider
        gr.Slider(label="Blood Pressure", minimum=80, maximum=200, step=1, value=120),  # Blood pressure as slider
        gr.Textbox(label="Symptoms Description", placeholder="Describe your symptoms...")  # Symptoms description as textbox
    ],
    outputs=[
        gr.Textbox(label="Diagnosis"),  # Diagnosis output
        gr.Textbox(label="Recommendation")  # Recommendation output
    ],
    title="Cardiovascular Risk Analysis",
    description="Enter your medical data and symptoms to get a risk assessment."
)

iface.launch()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9h3tmq3p
  JVM stdout: /tmp/tmp9h3tmq3p/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp9h3tmq3p/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_9glcct
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ff58260880a226164.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
train.head()  # Display the first few rows of the dataset


countries,year,Risk_Probability_both,Risk_Probability_Male,Risk_Probability_Female
Afghanistan,2019,35.3,34.4,36.2
Afghanistan,2018,35.4,34.7,36
Afghanistan,2017,35.5,35,35.9
Afghanistan,2016,35.6,35.1,36
Afghanistan,2015,35.6,35.4,35.7
Afghanistan,2014,35.7,35.7,35.7
Afghanistan,2013,36.2,36.2,36.1
Afghanistan,2012,36.6,36.8,36.3
Afghanistan,2011,37.1,37.5,36.8
Afghanistan,2010,37.8,38.3,37.4


In [ ]:
train = h2o.import_file("/content/data.csv")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
